In [1]:
import pandas as pd


In [2]:
# btc_av = pd.read_csv("df_BTC_alphavantage_01_Feb.csv")
# eth_av = pd.read_csv("df_ETH_alphavantage_01_Feb.csv")
reddit_c = pd.read_csv("reddit_comments.csv")
reddit_p = pd.read_csv("posts.csv")
twitter = pd.read_csv("tweeter_data_pandas.csv")
twitter = twitter.drop(columns=["link"])

In [3]:
columns = ["id", "date", "content", "likes", "title", "author", "link", "other_scores", "replies",
           "topics", "social"]


In [5]:
reddit_p.columns, reddit_c.columns

(Index(['id', 'title', 'author', 'created_utc', 'subreddit', 'score',
        'num_comments', 'permalink', 'url', 'selftext', 'over_18', 'is_video',
        'is_original_content', 'is_self', 'is_meta', 'is_crosspostable',
        'is_reddit_media_domain', 'is_robot_indexable', 'is_gallery',
        'processed', 'comments_processed'],
       dtype='object'),
 Index(['id', 'parent_id', 'link_id', 'subreddit', 'author', 'body', 'score',
        'created_utc'],
       dtype='object'))

## Standardizing dataframes

In [4]:
# Reddit posts

reddit_p = reddit_p.rename(columns={
    "created_utc": "date",
    "num_comments": "replies",
    "subreddit": "topics",
    "url": "link",
    "selftext": "content",
    "score": "likes",
})

# not present columns: other_scores, social
reddit_p["other_scores"] = ""
reddit_p["social"] = "reddit"
# merge inner to be used

In [5]:
# reddit_p["title"] = reddit_p["title"].astype("")
# reddit_p["author"] = reddit_p["author"].astype("string")
reddit_p["date"] = pd.to_datetime(reddit_p["date"], unit="s")

reddit_p.describe()

,date,likes,replies,over_18,is_video,is_original_content,is_self,is_meta,is_crosspostable,is_reddit_media_domain,is_robot_indexable,is_gallery,processed,comments_processed
count,221224,221224.000000,221224.000000,221224.000000,221224.000000,221224.000000,221224.000000,221224.000000,221224.0,221224.000000,221224.000000,221224.000000,243260.000000,221224.0
mean,2022-12-12 16:26:37.871298304,17.535276,6.686268,0.042776,0.011590,0.005103,0.480038,0.000127,0.0,0.103488,0.549244,0.020784,0.909414,1.0
min,2022-11-03 19:32:10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0
25%,2022-11-22 15:28:26.249999872,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.0
50%,2022-12-12 18:37:26.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.0
75%,2022-12-31 19:14:48,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.000000,1.0
max,2023-01-24 19:30:21,44662.000000,9593.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.0
std,NaN,256.479772,56.388480,0.202351,0.107032,0.071256,0.499603,0.011250,0.0,0.304596,0.497570,0.142662,0.287020,0.0


In [7]:
columns.sort()
z = list(twitter.columns)
z.sort()
print(columns)
print(z)
twitter["name"] = twitter["name"] + " " + twitter["username"]
twitter = twitter.rename(columns={
    "tweet_id": "id",
    "name": "author",
    "posted_time": "date",
    "hashtags": "topics",
    "tweet_url": "link"
})

twitter["topics"] = twitter["topics"][:-1] + "," + twitter["category"] + "]"  # todo: be fixed
twitter = twitter.drop(["category"], axis=1)
twitter["social"] = "twitter"
twitter["title"] = ""
twitter["other_scores"] = ""


['author', 'content', 'date', 'id', 'likes', 'link', 'other_scores', 'replies', 'social', 'title', 'topics']
['Unnamed: 0', 'category', 'content', 'hashtags', 'images', 'is_retweet', 'likes', 'mentions', 'name', 'posted_time', 'profile_picture', 'replies', 'retweets', 'tweet_id', 'tweet_url', 'username', 'videos']


In [8]:
# res = pd.merge(twitter, reddit_p, how="inner")
# res.head()
twitter["id"] = twitter["id"].astype("object")
twitter["replies"] = twitter["replies"].astype("float")
twitter["likes"] = twitter["likes"].astype("float")



In [9]:
# res = pd.merge(twitter, reddit_p, how="inner")
res = pd.concat([twitter, reddit_p], ignore_index=True, join="inner")


In [10]:
# Reddit posts

reddit_c = reddit_c.rename(columns={
    "created_utc": "date",
    # "num_comments": "replies",
    "subreddit": "topics",
    "link_id": "link",
    "body": "content",
    "score": "likes",
})

# not present columns: other_scores, social
reddit_c["other_scores"] = ""
reddit_c["replies"] = 0
reddit_c["social"] = "reddit"
reddit_c["title"] = ""

reddit_c["date"] = pd.to_datetime(reddit_c["date"], unit="s")

# merge inner to be used


In [18]:
twitter["date"] = pd.to_datetime(twitter["date"]).dt.tz_localize(None)


In [19]:
res = pd.concat([reddit_p, reddit_c], ignore_index=True, join="inner")
res.head()

,id,author,replies,likes,date,content,topics,link,social,title,other_scores
0,1345138664377094144,Tyler Winklevoss tyler,483.0,4273.0,2021-01-01 22:43:43,Zcash $ZEC is easily the most underrated crypt...,"[],crypto]",https://twitter.com/tyler/status/1345138664377...,twitter,,
1,1345136516729860107,Ming Solenya Napenstein MingGao26,15.0,254.0,2021-01-01 22:35:11,maybe the 19% rosneft stake failed so Gazprom ...,"[],crypto]",https://twitter.com/MingGao26/status/134513651...,twitter,,
2,1345127710943830018,Cointelegraph Cointelegraph,15.0,219.0,2021-01-01 22:00:12,Banking giant Wells Fargo finally bent the kne...,"['CryptoYearInReview'],crypto]",https://twitter.com/Cointelegraph/status/13451...,twitter,,
3,1345125794385944576,RYAN SΞAN ADAMS - rsa.eth RyanSAdams,19.0,306.0,2021-01-01 21:52:35,If you don’t have assets you’re screwed. \r\n\...,"[],crypto]",https://twitter.com/RyanSAdams/status/13451257...,twitter,,
4,1345100324705423360,Binance binance,1286.0,1697.0,2021-01-01 20:11:22,What are your 2021 #crypto predictions?,"['crypto'],crypto]",https://twitter.com/binance/status/13451003247...,twitter,,


In [20]:
print(res.columns, len(res.columns))
print(columns, len(columns))

Index(['id', 'author', 'replies', 'likes', 'date', 'content', 'topics', 'link',
       'social', 'title', 'other_scores'],
      dtype='object') 11
['author', 'content', 'date', 'id', 'likes', 'link', 'other_scores', 'replies', 'social', 'title', 'topics'] 11


In [24]:
res.dtypes

id                      object
author                  object
replies                float64
likes                  float64
date            datetime64[ns]
content                 object
topics                  object
link                    object
social                  object
title                   object
other_scores            object
dtype: object

In [25]:
res.to_csv("Aggregated_reddit_twitter.csv")